In [5]:
#cd gdrive/My\ Drive/
#cd ColabNotebooks/
cd DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master/


/content/gdrive/My Drive/ColabNotebooks/DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master


In [6]:
import utils_channels_last
from utils_channels_last import Vgg16BN


Using TensorFlow backend.


In [0]:
import numpy as np
from utils_channels_last import Vgg16BN
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model

## Classifying the Dog Breed Using (Transfer Leraning)
Here we are using  pretrained VGG16BN network .Since there are less no. of images per class.There is a high chance of overfitting so we use VGG16 network with Batch Normalization.

In [0]:
batch_size=64##This parameter should Be set according to your GPU/CPU capacity . Else Be Ready FOr Exhaust Error!
num_classes = 120
image_size = 224
num_channels = 3

## IMAGE DATA  AUGMENTATION 
Here we use data Augmentation as Described in the previous post.

In [11]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory('images', target_size=(224,224),
            class_mode='categorical', shuffle=True, batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory('images', target_size=(224,224),
            class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 20580 images belonging to 120 classes.
Found 20580 images belonging to 120 classes.


In [12]:
base_model = Vgg16BN(include_top=False).model
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# and a softmax layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

/content/gdrive/My Drive/ColabNotebooks/DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master/utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/gdrive/My Drive/ColabNotebooks/DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master/utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/gdrive/My Drive/ColabNotebooks/DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master/utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/gdrive/My Drive/ColabNotebooks/DOG-BREED-CLASSIFICATION-STANFORD-DOG-DATASET-master/utils_channels_last.py:58: UserWarning: Update your `Conv2D` call

In [17]:
print(train_generator)
model.fit_generator(train_generator, samples_per_epoch=train_generator.samples, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.samples)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=321, epochs=10, validation_steps=20580)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/10
320/321 [============================>.] - ETA: 1s - loss: 15.9497 - acc: 0.0085

KeyboardInterrupt: ignored

In [14]:
model.optimizer.lr = 1e6
model.fit_generator(train_generator, samples_per_epoch=train_generator.samples, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.samples)

AttributeError: ignored

In [0]:
model.save_weights('vgg_model_weights.h5')